In [1]:
import requests  # Library for fetching data from APIs
import pandas as pd  # For handling and analyzing data
import numpy as np  # For numerical operations
import pytz


In [2]:
# Machine Learning utilities
from sklearn.model_selection import train_test_split  # Splitting data into training and testing sets
from sklearn.preprocessing import LabelEncoder  # Converting categorical data into numerical values


In [3]:
# Models for classification and regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [4]:
# Metric for evaluating model performance
from sklearn.metrics import mean_squared_error  

In [5]:
# Handling date and time
from datetime import datetime, timedelta  

In [6]:
API_KEY = 'You_API_KEY'  # Replace with your actual API key
BASE_URL = 'https://api.openweathermap.org/data/2.5/'  # Base URL for making API requests

In [7]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"  # Construct the API request URL
    response = requests.get(url)  # Send the GET request to API
    data = response.json()
    print("API Response:", data)
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data.get('wind',{}).get('speed',0),
        'pressure': data['main']['pressure'],
        'wind_gust_speed': data.get('wind', {}).get('speed', 0)

    }

In [8]:
def prepare_data(data):
    le = LabelEncoder()  # Create a LabelEncoder instance
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # Define the feature variables and target variables
    x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]  # Feature variables
    y = data['RainTomorrow']  # Target variable

    return x, y, le  # Return feature variable, target variable, and the LabelEncoder

In [10]:
def train_rain_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)  # Train the model

    y_pred = model.predict(x_test)  # Make predictions on test set
    print("Mean Squared Error for Rain Model:", mean_squared_error(y_test, y_pred))

    return model

In [11]:
def prepare_regression_data(data, feature):
    x, y = [], []  # Initialize lists for feature and target values 
    
    for i in range(len(data) - 1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])

    x = np.array(x).reshape(-1, 1)
    y = np.array(y)

    return x, y

In [12]:
def train_regression_model(x, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x, y)
    return model

In [13]:
def predict_future(model, current_value):
    predictions = [current_value]
    for _ in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

In [14]:
def read_historical_data(file_path):
    return pd.read_csv(file_path)

In [18]:
def weather_view():
    city=input('Enter any city name:')
    current_weather=get_current_weather(city)
    print("Current Weather Data:", current_weather)


    #load historical data
    historical_data=read_historical_data('weather.csv')
    historical_data.columns = historical_data.columns.str.lower()
    print("Historical Data Columns:", historical_data.columns)
    print("Historical Data Columns:", historical_data.columns.tolist())




    #prepare and train the rain prediction model
    x,y,le=prepare_data( historical_data)
    rain_model= train_rain_model(x,y)

    #map wind direction to campass points
    wind_deg=current_weather['wind_gust_dir']%360
    compass_points=[
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75),("E", 78.75, 101.25),("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25),("SSE", 146.25, 168.75),("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75),("SW", 213.75, 236.25),("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25),("WNW", 281.25, 303.75),("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]
    compass_direction=next(point for point,start,end in compass_points if start<=wind_deg< end)
    compass_direction_encoded=le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    
    current_data = {
    'MinTemp': current_weather.get('temp_min', 0),
    'MaxTemp': current_weather.get('temp_max', 0),
    'windgustdir': current_weather.get('wind_gust_dir', -1),  # Use .get() to avoid KeyError
    'WindGustSpeed': current_weather.get('wind_gust_speed', 0),
    'Humidity': current_weather.get('humidity', 0),
    'Pressure': current_weather.get('pressure', 0),
    'Temp': current_weather.get('current_temp', 0),
}

    current_df=pd.DataFrame([current_data])

    #rain prediction
    rain_prediction=rain_model.predict(current_df)[0]

    #prepare regression model for temperature and humidity
    x_temp,y_temp=prepare_regression_data(historical_data,'Temp')
    x_hum,y_hum=prepare_regression_data(historical_data,'humidity')
    temp_model=train_regression_model(x_temp,y_temp)
    hum_model=train_regression_model(x_hum,y_hum)

    #predict future temperature and humidity
    future_temp=predict_future(temp_model,current_weather['Temp_min'])
    future_humidity=predict_future(hum_model,current_weather['humidity'])

    #prepare time for future predictions
    timezone=pytz.timezone('Asia/Karachi')
    now=datetime.now(timezone)
    next_hour=now + timedelta(hours=1)
    next_hour=next_hour.replace(minute=0,second=0,microsecond=0)

    future_times=[(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    #display results
    print(f"City:{city},{current_weather['country']}")
    print(f"Current Temperature:{current_weather['current_temp']}")
    print(f"Feels Like:{current_weather['feels_like']}")
    print(f"Minimum Temperature:{current_weather['temp_min']}°C")
    print(f"Maximum Temperature:{current_weather['temp_max']}°C")
    print(f"Humidity:{current_weather['humidity']}%")
    print(f"Weather Prediction:{current_weather['description']}")
    print(f"Rain Prediction:{'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Prediction:")
    for time,temp in zip(future_times,future_temp):
        print(f"{time}:{round(temp,1)}°C")

    print("\nFuture Humidity Prediction:")
    for time,humidity in zip(future_times,future_humidity):
        print(f"{time}:{round(humidity,1)}%")


weather_view()

API Response: {'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'base': 'stations', 'main': {'temp': 26.64, 'feels_like': 26.64, 'temp_min': 26.12, 'temp_max': 26.99, 'pressure': 1015, 'humidity': 81, 'sea_level': 1015, 'grnd_level': 1014}, 'visibility': 6000, 'wind': {'speed': 2.06, 'deg': 130}, 'clouds': {'all': 20}, 'dt': 1740246205, 'sys': {'type': 2, 'id': 2093220, 'country': 'IN', 'sunrise': 1740185922, 'sunset': 1740228386}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}
Current Weather Data: {'city': 'Chennai', 'current_temp': 27, 'feels_like': 27, 'temp_min': 26, 'temp_max': 27, 'humidity': 81, 'description': 'few clouds', 'country': 'IN', 'wind_gust_dir': 2.06, 'pressure': 1015, 'wind_gust_speed': 2.06}
Historical Data Columns: Index(['mintemp', 'maxtemp', 'windgustdir', 'windgustspeed', 'humidity',
       'pressure', 'temp', 'raintomorrow'],
      dtype='object')
Historical Da

KeyError: 'WindGustDir'